# CVDDivergence Strategy Analysis

This notebook provides an in-depth analysis of the CVDDivergence strategy, including its logic, indicators, and backtest results.

## Setup

### Change Working directory to repository root

In [1]:
from constants import set_cwd_to_project_root, CONFIG_DIR
from main import generate_config

generate_config("binance")

set_cwd_to_project_root()

### Configure Freqtrade environment

In [2]:
from freqtrade.configuration import Configuration


config = Configuration.from_files([CONFIG_DIR])

config.update(
    {
        "timeframe": "5m",
        "strategy": "CVDDivergence"
    }
)

# Pair to analyze - Only use one pair here
PAIR = "BTC/USDT:USDT"

In [3]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from constants import DATA_DIR

candles = load_pair_history(
    datadir=DATA_DIR / "binance",
    timeframe=config["timeframe"],
    pair=PAIR,
    data_format="feather",
    candle_type=CandleType.FUTURES,
)

# Confirm success
print(f"Loaded {len(candles)} rows of data for {PAIR} from {DATA_DIR}")
candles.head()

Loaded 77949 rows of data for BTC/USDT:USDT from F:\quantative-analysis\data


,date,open,high,low,close,volume
0,2024-01-01 00:00:00+00:00,42314.0,42437.2,42289.6,42437.1,1724.210
1,2024-01-01 00:05:00+00:00,42437.2,42474.1,42420.5,42446.8,994.003
2,2024-01-01 00:10:00+00:00,42446.8,42535.0,42445.2,42532.5,899.775
3,2024-01-01 00:15:00+00:00,42532.4,42603.2,42494.1,42494.1,1291.232
4,2024-01-01 00:20:00+00:00,42494.1,42533.1,42484.4,42509.4,418.463


## Load and run strategy

In [4]:
# Load strategy using values set above
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver

strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)

strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": PAIR})
df.describe()

,open,high,low,close,volume,max_delta,min_delta,bid,ask,delta,...,bull_poc_upper,bear_poc_lower,bear_poc_upper,atr,enter_long,enter_short,atr_stoploss_long,atr_stoploss_short,exit_long,exit_short
count,77949.000000,77949.000000,77949.000000,77949.000000,77949.000000,1153.000000,1153.000000,1153.000000,1153.000000,1153.000000,...,77949.000000,77949.000000,77949.00000,77935.000000,8.0,12.0,77935.000000,77935.000000,11663.0,12226.0
mean,59967.662478,60030.856862,59902.850450,59967.968266,986.058349,119.961145,-117.071254,424.177497,423.685794,-0.491703,...,58833.860199,58797.613522,58844.97786,128.006703,1.0,1.0,59715.097593,60227.124405,1.0,1.0
std,8664.717434,8673.167050,8655.938569,8664.514976,1518.518217,239.786984,207.156362,514.070818,551.360557,303.569234,...,8502.229830,8505.172908,8509.39549,77.712073,0.0,0.0,8628.866498,8698.022935,0.0,0.0
min,38588.000000,38664.800000,38545.000000,38587.900000,21.015000,-4.734000,-2867.582000,40.676000,28.915000,-2777.391000,...,38619.174000,38587.900000,38619.17400,9.537812,1.0,1.0,38305.554589,38856.762043,1.0,1.0
25%,56714.100000,56788.300000,56651.500000,56714.600000,302.570000,11.545000,-135.276000,155.460000,153.662000,-89.743000,...,54889.638000,54735.700000,54889.63800,76.733256,1.0,1.0,56434.758545,57009.310797,1.0,1.0
50%,62349.800000,62434.300000,62261.400000,62349.900000,544.831000,48.157000,-54.776000,259.100000,263.854000,-3.815000,...,60901.930000,60860.400000,60901.93000,109.267652,1.0,1.0,62046.940067,62675.126487,1.0,1.0
75%,66563.000000,66625.900000,66496.100000,66563.000000,1068.261000,142.213000,-12.481000,472.862000,461.927000,90.340000,...,65221.580000,65180.700000,65249.88000,160.092537,1.0,1.0,66277.803698,66827.143750,1.0,1.0
max,73698.500000,73881.400000,73641.500000,73698.500000,44889.025000,3620.511000,5.219000,5796.983000,7313.838000,3425.524000,...,71475.692000,71430.500000,71475.69200,877.010063,1.0,1.0,73411.005269,74101.017397,1.0,1.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77949 entries, 0 to 77948
Data columns (total 41 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   date                    77949 non-null  datetime64[ns, UTC]
 1   open                    77949 non-null  float64            
 2   high                    77949 non-null  float64            
 3   low                     77949 non-null  float64            
 4   close                   77949 non-null  float64            
 5   volume                  77949 non-null  float64            
 6   trades                  1153 non-null   object             
 7   orderflow               1153 non-null   object             
 8   imbalances              1153 non-null   object             
 9   stacked_imbalances_bid  267 non-null    object             
 10  stacked_imbalances_ask  214 non-null    object             
 11  max_delta               1153 non-null   float6

### Display the trade details

In [6]:
# Report results
print(f"Generated {df['enter_long'].sum()} long entry signals")
print(f"Generated {df['enter_short'].sum()} short entry signals")
data = df.set_index("date", drop=False)

Generated 8.0 long entry signals
Generated 12.0 short entry signals


## Load backtest results

In [7]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
from constants import BACKTEST_RESULTS_DIR

# Load backtest statistics
stats = load_backtest_stats(BACKTEST_RESULTS_DIR)

strategy = "CVDDivergence"
# Example usages:
print("Results per pair:")
print(stats["strategy"][strategy]["results_per_pair"])
print("\nPairlist:")
print(stats["strategy"][strategy]["pairlist"])
print(f"\nMarket change: {stats['strategy'][strategy]['market_change']}")
print(f"Drawdown start: {stats['strategy'][strategy]['drawdown_start']}")
print(f"Drawdown end: {stats['strategy'][strategy]['drawdown_end']}")

# Load backtested trades as dataframe
trades = load_backtest_data(BACKTEST_RESULTS_DIR)

if len(trades) > 0:
    print("\nExit reasons per pair:")
    print(trades.groupby("pair")["exit_reason"].value_counts())

Results per pair:
[{'key': 'BTC/USDT:USDT', 'trades': 1, 'profit_mean': -0.020599915181841476, 'profit_mean_pct': -2.06, 'profit_sum': -0.020599915181841476, 'profit_sum_pct': -2.06, 'profit_total_abs': -20.33600742, 'profit_total': -0.02033600742, 'profit_total_pct': -2.03, 'duration_avg': '1:30:00', 'wins': 0, 'draws': 0, 'losses': 1, 'winrate': 0.0}, {'key': 'TOTAL', 'trades': 1, 'profit_mean': -0.020599915181841476, 'profit_mean_pct': -2.06, 'profit_sum': -0.020599915181841476, 'profit_sum_pct': -2.06, 'profit_total_abs': -20.33600742, 'profit_total': -0.02033600742, 'profit_total_pct': -2.03, 'duration_avg': '1:30:00', 'wins': 0, 'draws': 0, 'losses': 1, 'winrate': 0.0}]

Pairlist:
['BTC/USDT:USDT']

Market change: 0.12001015572105617
Drawdown start: 1970-01-01 00:00:00+00:00
Drawdown end: 1970-01-01 00:00:00+00:00

Exit reasons per pair:
pair           exit_reason
BTC/USDT:USDT  VWAP_Lower     1
Name: count, dtype: int64


## Plotting daily profit / equity line

In [8]:
import pandas as pd
import plotly.express as px

strategy_stats = stats["strategy"][strategy]

equity = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
equity["equity_daily"] = equity["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily", title="CVDDivergence Strategy - Equity Curve")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['date', 'open', 'high', 'low', 'close', 'volume', 'trades', 'orderflow', 'imbalances', 'stacked_imbalances_bid', 'stacked_imbalances_ask', 'max_delta', 'min_delta', 'bid', 'ask', 'delta', 'total_trades', 'day', 'cvd', 'pivot_low', 'pivot_high', 'cvd_ma', 'divergence', 'vwap', 'vwap_upper_1', 'vwap_lower_1', 'vwap_upper_2', 'vwap_lower_2', 'bull_poc_lower', 'bull_poc_upper', 'bear_poc_lower', 'bear_poc_upper', 'atr', 'enter_tag', 'enter_long', 'enter_short', 'atr_stoploss_long', 'atr_stoploss_short', 'exit_tag', 'exit_long', 'exit_short'] but received: dates

## Plot results

In [9]:
from freqtrade.plot.plotting import generate_candlestick_graph

from_date = data["delta"].dropna().index.min()
to_date = data["delta"].dropna().index.max()

# Filter data for a specific date range
data_filtered = data[from_date:to_date].drop(columns=["exit_short", "exit_long"])

# Generate candlestick graph with indicators for trade entry and losses
graph = generate_candlestick_graph(
    pair=PAIR,
    data=data_filtered,
    trades=trades,
    indicators1=["vwap", "vwap_lower_2", "vwap_upper_2", "bull_poc_upper", "bull_poc_lower", "bear_poc_lower", "bear_poc_upper"],
    indicators2=["cvd_ma"]
)

# Render graph in a separate window
graph.show(renderer="browser")